In [1]:
from grand import lig_samplers as ls

Warning on use of the timeseries module: If the inherent timescales of the system are long compared to those being analyzed, this statistical inefficiency may be an underestimate.  The estimate presumes the use of many statistically independent samples.  Tests should be performed to assess whether this condition is satisfied.   Be cautious in the interpretation of the data.

****** PyMBAR will use 64-bit JAX! *******
* JAX is currently set to 32-bit bitsize *
* which is its default.                  *
*                                        *
* PyMBAR requires 64-bit mode and WILL   *
* enable JAX's 64-bit mode when called.  *
*                                        *
* This MAY cause problems with other     *
* Uses of JAX in the same code.          *
******************************************



In [2]:
import openmm
import openmm.app
import openmm.unit

from openmmtools.integrators import VelocityVerletIntegrator

import numpy as np

In [3]:
pdb = openmm.app.PDBFile('gcmc-ghosts.pdb')

In [4]:
ff = openmm.app.ForceField('amber14-all.xml', 'amber14/tip3pfb.xml','phenol_openff.xml')

In [5]:
system = ff.createSystem(pdb.topology)

In [6]:
id(system)

127467632177280

In [7]:
system.addForce(openmm.CMMotionRemover())

5

In [8]:
frag_res_ids = []
for resid, residue in enumerate(pdb.topology.residues()):
    if residue.name == "UNK":
        frag_res_ids.append(resid)

In [9]:
sampler = ls.BaseGCMCSampler(system,pdb.topology,298*openmm.unit.kelvin,frag_res_ids,overwrite=True)

In [10]:
integrator = openmm.LangevinIntegrator(
        298.0 * openmm.unit.kelvin, 1.0 / openmm.unit.picosecond, 2.0 * openmm.unit.femtosecond
    )

In [11]:
integrator = VelocityVerletIntegrator(
    1.0 * openmm.unit.femtosecond
)

In [12]:
integrator = openmm.VerletIntegrator(
    1.0 * openmm.unit.femtosecond
)

In [13]:
reporter = openmm.app.DCDReporter("phenol_sacp.dcd", reportInterval=100)

In [14]:
sampler.initialize(100, pdb.positions,integrator,reporter)

In [15]:
sampler.insert(sampler.lig_atom_ids[0])

Quantity(value=[Vec3(x=-0.6778, y=-0.1424, z=0.42000000000000004), Vec3(x=-0.6878000000000001, y=-0.0708, z=0.2896), Vec3(x=-0.5557000000000001, y=-0.084, z=0.2138), Vec3(x=-0.46399999999999997, y=-0.1504, z=0.2579), Vec3(x=-0.5778, y=-0.1527, z=0.4462), Vec3(x=-0.7214, y=-0.23650000000000004, z=0.4112), Vec3(x=-0.7273000000000001, y=-0.0879, z=0.4933), Vec3(x=-0.7677, y=-0.11399999999999999, z=0.23090000000000002), Vec3(x=-0.7085, y=0.033600000000000005, z=0.3073), Vec3(x=-0.5452, y=-0.0212, z=0.0999), Vec3(x=-0.41890000000000005, y=-0.0302, z=0.0213), Vec3(x=-0.31970000000000004, y=0.07440000000000001, z=0.0717), Vec3(x=-0.3252, y=0.1898, z=0.0342), Vec3(x=-0.44900000000000007, y=-0.0048000000000000004, z=-0.1264), Vec3(x=-0.32160000000000005, y=-0.0141, z=-0.2068), Vec3(x=-0.27799999999999997, y=-0.1383, z=-0.2544), Vec3(x=-0.24740000000000004, y=0.10149999999999999, z=-0.23420000000000002), Vec3(x=-0.16010000000000002, y=-0.147, z=-0.32930000000000004), Vec3(x=-0.1295, y=0.0927, z=

In [16]:
sampler.real_lig_res_ids

[]

In [17]:
f = open('real_res_ids.txt','w')
for n in range(100):
    sampler.move()
    sampler.reporter.report(sampler.simulation,sampler.context.getState(getPositions=True,getEnergy=True))
    f.write(','.join(sampler.real_lig_res_ids)+'\n')
    print(sampler.N)

0
-17361.38983440399 kJ/mol
-19145.97930908203 kJ/mol
1
-17324.67267394066 kJ/mol
-19099.126812934875 kJ/mol
2
2
-17286.999278068542 kJ/mol
-19002.687496185303 kJ/mol
3
3
3
987650.1464595795 kJ/mol
-18956.044479370117 kJ/mol
4
987698.1493091583 kJ/mol
-18935.644744873047 kJ/mol
5
5
5
5250342264110.902 kJ/mol
-18962.939098358154 kJ/mol
6
6
6
6
6
5250342264144.757 kJ/mol
-18691.49487966299 kJ/mol
6
6
6
6
6
6
6
6
5250360767612.082 kJ/mol
-19015.74968147278 kJ/mol
7
5250360767648.821 kJ/mol
-18980.733449935913 kJ/mol
8
5250366010569.747 kJ/mol
-18878.37826538086 kJ/mol
9
9
5250366010592.237 kJ/mol
-18852.644855499268 kJ/mol
10
10
5250366010588.96 kJ/mol
-18814.650415420532 kJ/mol
11
11
11
5250373998860.077 kJ/mol
-18919.96069908142 kJ/mol
12
12
5250373998902.2295 kJ/mol
-18883.384616851807 kJ/mol
13
13
5250373998940.36 kJ/mol
-18843.163959503174 kJ/mol
14
5250373999020.456 kJ/mol
-18809.690732955933 kJ/mol
15
15
5250373999059.364 kJ/mol
-18882.315671920776 kJ/mol
16
5250373999101.719 kJ/mo

In [18]:
np.random.rand(3) * (sampler.max_dimension - sampler.min_dimension)._value * openmm.unit.nanometer + sampler.min_dimension

Quantity(value=array([ 0.20788215,  1.36335546, -0.27255203]), unit=nanometer)

In [19]:
np.random.rand(3)

array([0.76953109, 0.01319799, 0.33902336])

In [20]:
np.multiply(np.random.rand(3) , (sampler.max_dimension - sampler.min_dimension))

ValueError: setting an array element with a sequence.

In [ ]:
openmm.Vec3(*np.multiply(np.random.rand(3) , (sampler.max_dimension - sampler.min_dimension))) #+ sampler.min_dimension

ValueError: setting an array element with a sequence.

In [ ]:
sampler.N

312

In [ ]:
sampler.lig_params

defaultdict(list,
            {138: [Quantity(value=-0.18487000580017382, unit=elementary charge),
              Quantity(value=0.34806468869450646, unit=nanometer),
              Quantity(value=0.3635030558377792, unit=kilojoule/mole)],
             139: [Quantity(value=-0.09450999762003238, unit=elementary charge),
              Quantity(value=0.34806468869450646, unit=nanometer),
              Quantity(value=0.3635030558377792, unit=kilojoule/mole)],
             140: [Quantity(value=-0.1652700018424254, unit=elementary charge),
              Quantity(value=0.34806468869450646, unit=nanometer),
              Quantity(value=0.3635030558377792, unit=kilojoule/mole)],
             141: [Quantity(value=-0.09450999762003238, unit=elementary charge),
              Quantity(value=0.34806468869450646, unit=nanometer),
              Quantity(value=0.3635030558377792, unit=kilojoule/mole)],
             142: [Quantity(value=-0.18487000580017382, unit=elementary charge),
              Quantity

In [ ]:
system.getForce(1).getParticleParameters(138)

[Quantity(value=0.0, unit=elementary charge),
 Quantity(value=0.0, unit=nanometer),
 Quantity(value=0.0, unit=kilojoule/mole)]

In [ ]:
sampler.real_lig_res_ids

['263',
 '423',
 '382',
 '621',
 '547',
 '182',
 '221',
 '454',
 '491',
 '174',
 '837',
 '596',
 '594',
 '891',
 '247',
 '785',
 '295',
 '440',
 '815',
 '389',
 '540',
 '45',
 '918',
 '758',
 '772',
 '501',
 '606',
 '139',
 '484',
 '644',
 '220',
 '997',
 '393',
 '746',
 '83',
 '732',
 '602',
 '607',
 '226',
 '183',
 '816',
 '290',
 '28',
 '762',
 '39',
 '23',
 '456',
 '399',
 '578',
 '307',
 '169',
 '145',
 '1005',
 '920',
 '987',
 '819',
 '479',
 '191',
 '32',
 '913',
 '924',
 '571',
 '716',
 '620',
 '678',
 '864',
 '244',
 '275',
 '523',
 '565',
 '124',
 '975',
 '86',
 '305',
 '593',
 '669',
 '555',
 '592',
 '122',
 '520',
 '525',
 '953',
 '779',
 '632',
 '848',
 '773',
 '283',
 '14',
 '24',
 '806',
 '13',
 '710',
 '743',
 '333',
 '130',
 '650',
 '656',
 '611',
 '580',
 '460',
 '488',
 '836',
 '901',
 '639',
 '358',
 '1003',
 '413',
 '821',
 '165',
 '72',
 '962',
 '286',
 '867',
 '672',
 '572',
 '214',
 '126',
 '463',
 '219',
 '161',
 '921',
 '373',
 '649',
 '966',
 '150',
 '210',
 